En este notebook se continuará trabajando con el dataset **"g_relative_results_v3_parquet"** para su transformación. De resultado se obtendrá una tabla para su posterior carga a una base de datos.

Como primer paso voy a importar el dataset.

In [184]:
# Importo librería necesaria
import pandas as pd

df = pd.read_parquet(r"C:\Users\Juli\Desktop\g_relative_results_v3.parquet")

# Muestra las primeras filas del archivo para verificar su contenido
df.head()

,id_negocio,relative_results
0,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,0x89c25896a272f571:0x591dc6b82a5c9f69
1,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,0x89c259aa95e35ab1:0xe5bc3090ddbb7303
2,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,0x89c2f6738352c9b1:0xbd4236721db97644
3,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,0x89c2f63cfaa99b63:0x9d122f0cb09de6ec
4,0x89c2f6163ac2864f:0xa4213a2f66c15d8f,0x89c2607f4b59e67f:0x9101ba64ab5e2d9c


In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69812 entries, 0 to 69811
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_negocio        69812 non-null  object
 1   relative_results  67943 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [187]:
# Realizo una copia del dataframe
df_copia = df.copy()

Voy a realizar una búsqueda de **valores nulos.**

In [188]:
# Cuento los nulos en cada fila
cant_nulos = df_copia.isnull().sum()
print(cant_nulos)

id_negocio             0
relative_results    1869
dtype: int64


In [189]:
# Muestro las filas que contienen algún valor nulo
df_copia[df.isnull().any(axis=1)]

,id_negocio,relative_results
27,0x80c33190ba5119bf:0x1a8b79348a959bd4,None
66,0x8637078cc7ea293f:0x56780adebdd561a7,None
67,0x89ccb14964182235:0x117c242f50e9def1,None
72,0x8083fd560ecc3d1b:0x83822a243b78ff83,None
80,0x8640db0e8489dbdb:0x4ce70ca2576c9005,None
...,...,...
69552,0x88e76353c905db17:0x3773d93435328b7e,None
69595,0x89c4c5b181efd00d:0xfa13b3f3ce1e08f2,None
69726,0x80c2d9df7154560d:0xf714b3bfa2b48916,None
69752,0x864e994da76408d3:0xbc350830290d819c,None


In [190]:
# Calculo el porcentaje de nulos de la columna relative_results
nulos_relative_results = df_copia['relative_results'].isnull().sum()
total_filas = len(df)
porcentaje_nulos = (nulos_relative_results / total_filas) * 100

print(porcentaje_nulos)

2.6771901678794476


Podemos observar que la cantidad de nulos es baja y debido a que uno de los objetivos propuestos en este proyecto es identificar las mejores ubicaciones para nuevos locales basándose en la densidad de competidores y la demanda de usuarios, he decidido dejarlos. Son datos valiosos para el análisis. Voy a reemplazar los nulos por "sin competencia".

In [191]:
# Desactivo SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [192]:
# Reemplazo valores de tipo None o el texto "None" por "sin competencia"
df_copia['relative_results'] = df_copia['relative_results'].replace([None, "None"], "sin competencia")

In [193]:
# Filtro y muestro las filas donde relative_results es "sin competencia"
sin_competencia_filas = df_copia[df_copia['relative_results'] == "sin competencia"]
sin_competencia_filas

,id_negocio,relative_results
27,0x80c33190ba5119bf:0x1a8b79348a959bd4,sin competencia
66,0x8637078cc7ea293f:0x56780adebdd561a7,sin competencia
67,0x89ccb14964182235:0x117c242f50e9def1,sin competencia
72,0x8083fd560ecc3d1b:0x83822a243b78ff83,sin competencia
80,0x8640db0e8489dbdb:0x4ce70ca2576c9005,sin competencia
...,...,...
69552,0x88e76353c905db17:0x3773d93435328b7e,sin competencia
69595,0x89c4c5b181efd00d:0xfa13b3f3ce1e08f2,sin competencia
69726,0x80c2d9df7154560d:0xf714b3bfa2b48916,sin competencia
69752,0x864e994da76408d3:0xbc350830290d819c,sin competencia


Voy a realizar una búsqueda de valores **duplicados.**

In [194]:
# Identifico y muestro filas duplicadas
duplicados = df_copia[df_copia.duplicated(keep=False)]
print(duplicados)

Empty DataFrame
Columns: [id_negocio, relative_results]
Index: []


No se han encontrado filas duplicadas.

Revisaré si los tipos de datos de las columnas son los que necesito de acuerdo al modelo relacional creado para la base de datos.

In [196]:
print(df_copia.dtypes)

id_negocio          object
relative_results    object
dtype: object


Exportaré la tabla ya que está lista para ser utilizada en nuestra base de datos.

In [197]:
df_copia.to_parquet(r"C:\Users\Juli\Desktop\g_relative_results_v4.parquet")